%%

In [53]:
import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import glob
# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook, tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
from lxml import etree, html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, extract_title_and_name, get_recent_file
from unidecode import unidecode

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [54]:
from bs4 import BeautifulSoup

In [55]:
# aggregate chamber info list
mo_com_dfs = {}

In [56]:
def create_pk2(state, chamber, district, mls):
    
    if not re.search(r'^[A-Z]{2}$', state):
        
        error = 'func error: state is wrong'
        return print(error)
        

    state_code = state_coding.get(state)
    
    if not re.search(r'[Hh]ouse|[Ss]enate', chamber):
        error = 'func error: chamber wrong'
        return print(error)
    elif re.search(r'[Hh]ouse',chamber):
        chamber_code = 0
    else:
        chamber_code = 1

    if not re.search(r'^\d+$', district):
        return print('func error: district is more than just numbers')
    else:
        dist_code = str(district).strip().zfill(3)

    if mls == 'y':
        print('im not set up for that yet')
    else:
        seat = '00'

    fpk = int(f'{state_code}{chamber_code}{dist_code}{seat}')
    return fpk

    

# Senate

In [66]:
file = r'c:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\MO_senate.xlsx'
mo_senate = pd.read_excel(file)
# print(mo_senate.columns)
mo_senate = mo_senate.rename(columns={'com name': 'com_name'})
print(mo_senate.columns)



Index(['chamber', 'com_name', 'name', 'role', 'district'], dtype='object')


In [58]:
mo_com_dfs['senate'] = mo_senate

In [59]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


# House
download csv files from site and put into a folder shown below

In [60]:
os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\to scrape\MO')
house_files = glob.glob(r'*')
print(*house_files, sep = '\n')



Budget_comm.csv
elementary_secondary_education_comm.csv
higher_ed_and_workforce_comm.csv
join_comm_on_education.csv
subcommittee_on_app_edu_comm.csv
ways_and_means_comm.csv


In [74]:
#compile and narrow files
house_dfs = []
for file in house_files:
    df = pd.read_csv(file)
    print(df.columns)
    print(df.head(2).to_string())

    df = df.loc[:,['FirstName', 'LastName','District', 'cp']]
    df['name'] = df['FirstName'] + ' ' + df['LastName']
    df['name'] = df['name'].str.replace('Rep.', '').str.strip()
    df['CommitteeName'] = file.replace('_', ' '). replace('.csv', '').title()
    df['chamber'] = 'house'
    df = df.rename(columns={'District': 'district', 'cp':'role', 'CommitteeName': 'com_name'})
    df = df.loc[:,['chamber', 'com_name', 'name', 'role', 'district']]
    
    house_dfs.append(df)



Index(['FirstName', 'LastName', 'LeadershipPositionCurrent', 'Party',
       'CapitolFax', 'Phone', 'Room', 'District', 'cp', 'CommitteeName'],
      dtype='object')
     FirstName  LastName                           LeadershipPositionCurrent       Party    CapitolFax         Phone   Room  District          cp CommitteeName
0    Rep. Dirk    Deaton                                                      Republican                573-751-9801    309       159       Chair        Budget
1  Rep. Bishop  Davidson                                                      Republican                573-751-2381  306-B       130  Vice-Chair        Budget
Index(['FirstName', 'LastName', 'LeadershipPositionCurrent', 'Party',
       'CapitolFax', 'Phone', 'Room', 'District', 'cp', 'CommitteeName'],
      dtype='object')
   FirstName   LastName                           LeadershipPositionCurrent       Party    CapitolFax         Phone   Room  District          cp                       CommitteeName
0    Re

In [ ]:
mo_house_coms = pd.concat(house_dfs).reset_index(drop=True)
# print(list(set(house_coms['role'].to_list())))
mo_house_coms

In [ ]:
mo_coms_df = pd.concat([mo_house_coms,mo_senate]).reset_index(drop=True)
mo_coms_df




#clean and assign

In [84]:
#assign fpk

tqdm.pandas()


lens = []
mo_coms_df['fpk'] = np.nan 

    # for row in mo_senate.itertuples(index=True):
for row in tqdm_notebook(mo_coms_df.itertuples(index=True),total=len(mo_coms_df)):
    

    fpk = create_pk2("MO", row.chamber, str(row.district), 'n')
    print(fpk)

    mo_coms_df.loc[row.Index,'fpk'] = fpk



fpk_pop = mo_coms_df.pop('fpk')
mo_coms_df.insert(0,"fpk", fpk_pop)


  0%|          | 0/125 [00:00<?, ?it/s]

34015900
34013000
34013500
34012900
34015700
34014900
34013700
34000200
34002900
34015800
34002700
34006500
34013600
34012600
34008200
34000600
34004400
34012400
34007800
34013100
34007300
34000500
34000400
34004500
34002000
34008400
34004800
34012300
34014700
34012800
34002200
34000600
34011900
34004500
34016000
34007000
34009000
34006300
34014000
34012000
34000800
34003800
34012700
34011700
34016300
34009900
34004400
34003900
34015500
34005200
34014100
34006800
34008700
34011100
34001600
34006200
34013600
34001700
34005800
34002500
34007300
34006100
34001100
34006800
34013300
34013900
34010600
34011600
34013700
34003200
34007600
34013000
34000600
34016300
34005200
34006600
34003100
34002200
34003300
34001700
34000200
34001900
34013000
34000600
34004500
34013700
34000200
34004400
34007300
34011800
34005600
34008400
34003200
34008800
34010700
34007500
34006400
34003400
34011600
34103000
34101200
34102500
34101600
34100800
34102800
34101000
34100300
34103300
34100400
34101700
34100900
3

,fpk,chamber,com_name,name,role,district
0,34015900.0,house,Budget Comm,Dirk Deaton,Chair,159
1,34013000.0,house,Budget Comm,Bishop Davidson,Vice-Chair,130
2,34013500.0,house,Budget Comm,Betsy Fogle,Ranking Minority Member,135
3,34012900.0,house,Budget Comm,John Black,NaN,129
4,34015700.0,house,Budget Comm,Mitch Boggs,NaN,157
...,...,...,...,...,...,...
120,34101100.0,Senate,"Families, Senior, and Health",Senator Nicola,Vice-Chair,11
121,34103100.0,Senate,"Families, Senior, and Health",Senator Brattin,Member,31
122,34103300.0,Senate,"Families, Senior, and Health",Senator Hudson,Member,33
123,34100700.0,Senate,"Families, Senior, and Health",Senator Lewis,Member,7


In [86]:
#set up
ref_path = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\key_creation\2025"
leg_ref =pd.read_excel(get_recent_file("*", ref_path))
leg_ref = leg_ref[leg_ref['state abbreviation'] == "MO"]
# print(leg_ref.columns)
# print(leg_ref.head())
# leg_ref


In [87]:
def get_subset(df, fpk_col, fpk):
    df_new = df[df[fpk_col] == fpk]
    return df_new

In [ ]:
for row in mo_senate.itertuples(index=False):
    print(row)
    if row.chamber == 'Senate':
        chamber_code

Pandas(chamber='Senate', _1='Appropiations', name='Senator Hough', role='Chair', district=30)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Black', role='Vice-Chair', district=12)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Bean', role='Member', district=25)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Brown', role='Member', district=16)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Cierpiot', role='Member', district=8)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Crawford', role='Member', district=28)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Fitzwater', role='Member', district=10)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Henderson', role='Member', district=3)
Pandas(chamber='Senate', _1='Appropiations', name='Senator Hudson', role='Member', district=33)
Pandas(chamber='Senate', _1='Appropiations', name='Senator May', role='Member', district=4)
Pandas(chamber='Senate', _1='Appropiat

In [ ]:
#checking that all are assigned correctly
lens = []
for row in mo_coms_df.itertuples(index=False):
    sub_df = get_subset(leg_ref, 'full_pk', int(row.fpk))
    lens.append(len(sub_df))
    if len(sub_df) >1:
        print(row)
        print(f'sub df :')
        for row in sub_df.itertuples(index=False):
            print(row)
    elif len(sub_df) <1:
        print(row)
        print(sub_df)
        
    # print(sub_df)
print(list(set(lens)))


"""
if this returns anything other than [1] that means that the fpk match isnt fully clean
"""

[1]


'\nif this returns anything other than [1] that means that the fpk match isnt fully clean\n'

In [91]:
#export
#export
com_folder = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\full_pk_assigned'
file_name = os.path.join(com_folder,"mo_com_info.xlsx")
mo_coms_df.to_excel(file_name, index=False)